In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
os.chdir('../') # set the working directory to the root of the project
from symai import *
from IPython.display import display
from examples.sql import SQL
sql = SQL()

## SQL Example

The `SQL` expression is defined and passes as a global context the syntax of the SQL language. The SQL expression is then used to generate queries based on the given context. We can then use the generated queries to get data from a database.

The following query is sent to the neural computation engine and creates a query based on the given context:

In [2]:
val = None
def _fun(x):
    global val
    val = x
Symbol._metadata.input_handler = _fun

In [ ]:
Symbol('I have this class').translate('German')

In [3]:
q = sql('Select all users above the age of 30 from the Members table.'); display(q)

<class 'examples.sql.SQL'>(value=SELECT * FROM Members WHERE age > 30)

We can now try to further manipulate the result by asking the model to incorporate additional information, such as filtering to a specific time range:

In [4]:
res = q << 'and limit the query to the last 10 minutes'
display(res)

<class 'symai.symbol.Symbol'>(value=SELECT * FROM Members WHERE age > 30 AND timestamp > NOW() - INTERVAL 10 MINUTE;)

We can also try to remove unwanted fields from the generated query. Notice how the model tries to remove not only the given statement but attributes associated with them:

In [7]:
res -= ' AND ...'
display(res)

<class 'symai.symbol.Symbol'>(value=SELECT * FROM Members WHERE age > 30)

And we can now even convert our query to other familiar domain specific languages, such as `SQL` or `ActiveRecord`:

In [9]:
sql_res = res.convert("ActiveRecord")
display(sql_res)

<class 'symai.symbol.Symbol'>(value=```ruby
Member.where("age > ?", 30)
```)

To wrap things up, we might want to go back full circle and ask the model to generate again the explanation based on the given query:

In [5]:
answer_doc = res.query("What does this query do?")
display(answer_doc)

<class 'symai.symbol.Symbol'>(value=The given SQL query retrieves all records from a table named `Members` where two conditions are met:

1. The `age` column of a member record must be greater than 30.
2. The `timestamp` column of a member record must be more recent than (or within) the last 10 minutes from the current time.

More specifically:

- `SELECT *` indicates that it will fetch all columns for the qualifying rows.
- `FROM Members` specifies the table from which to fetch the data.
- `WHERE` introduces the conditions to filter which rows should be returned.
- `age > 30` filters the records to only include those where the age column value is greater than 30.
- `AND` combines multiple conditions, meaning a record must meet all the specified conditions to be included.
- `timestamp > NOW() - INTERVAL 10 MINUTE` checks for records with a timestamp within the last 10 minutes. `NOW()` is a function that returns the current datetime. `INTERVAL 10 MINUTE` specifies a time interval of 10 minutes. By subtracting this interval from the current time, the query creates a datetime value that is 10 minutes before now. The condition `timestamp >` selects all records where the `timestamp` is more recent than that calculated time.

This query would be useful for identifying active, older members (above 30 years old) who have engaged with the system or performed an action that updates their `timestamp` within the last 10 minutes.)

Ask it in natural language to modify the query:

In [11]:
answer = res.query("How can you limit the number of results to 30 for an SQL query?")
display(answer)

<class 'symai.symbol.Symbol'>(value=To limit the number of results to 30 for an SQL query, you can use the `LIMIT` clause in your SQL statement. The `LIMIT` clause restricts the number of rows returned by the query. Here's how you modify your given SQL query to limit the results to 30:

```sql
SELECT * FROM Members WHERE age > 30 LIMIT 30;
```

This query will retrieve a maximum of 30 records from the 'Members' table where the 'age' column value is greater than 30.)

Even translate the explanation to a different language on the fly:

In [6]:
locale = Symbol(answer_doc).translate('German')
display(locale)

<class 'symai.symbol.Symbol'>(value=It seems like you haven't specified the target language for the translation. Please provide the target language and I will be happy to assist you with the translation.)

In [7]:
val[0]

[{'role': 'system',
  'content': '[INSTRUCTION]\nTranslate the following text into {}:\n'},
 {'role': 'user',
  'content': 'The given SQL query retrieves all records from a table named `Members` where two conditions are met:\n\n1. The `age` column of a member record must be greater than 30.\n2. The `timestamp` column of a member record must be more recent than (or within) the last 10 minutes from the current time.\n\nMore specifically:\n\n- `SELECT *` indicates that it will fetch all columns for the qualifying rows.\n- `FROM Members` specifies the table from which to fetch the data.\n- `WHERE` introduces the conditions to filter which rows should be returned.\n- `age > 30` filters the records to only include those where the age column value is greater than 30.\n- `AND` combines multiple conditions, meaning a record must meet all the specified conditions to be included.\n- `timestamp > NOW() - INTERVAL 10 MINUTE` checks for records with a timestamp within the last 10 minutes. `NOW()` is

Fixing the query on the fly if something goes wrong or the user quickly wants to adapt a query:

In [21]:
sql.adapt(context="""Explanation: Never allow SELECT *, always use LIMIT to a max of x <= 50 entries, where x is the user specified limit.""");

In [22]:
res = sql('Select all users above the age of 30 from the Members table.')
display(res)

<class 'examples.sql.SQL'>(value=SELECT * FROM Members WHERE age > 30 LIMIT 50)

In [23]:
sql.clear();

In [24]:
res = sql('Select all users above the age of 30 from the Members table.')
display(res)

<class 'examples.sql.SQL'>(value=SELECT * FROM Members WHERE age > 30)